## **IMPORTS AND SETUP**

In [ ]:
#allows the use of SQL
import duckdb
#data manipulation
import pandas as pd
#makes use of python os
import os

## **GETTING IMDB DATASETS**

In [ ]:
#download and load the imdb dataset
!wget https://datasets.imdbws.com/title.principals.tsv.gz
!wget https://datasets.imdbws.com/title.akas.tsv.gz
!wget https://datasets.imdbws.com/title.crew.tsv.gz
!wget https://datasets.imdbws.com/title.episode.tsv.gz
!wget https://datasets.imdbws.com/title.basics.tsv.gz
!wget https://datasets.imdbws.com/title.ratings.tsv.gz
!wget https://datasets.imdbws.com/name.basics.tsv.gz

--2025-10-20 23:57:30--  https://datasets.imdbws.com/title.principals.tsv.gz
Resolving datasets.imdbws.com (datasets.imdbws.com)... 13.249.98.61, 13.249.98.91, 13.249.98.73, ...
Connecting to datasets.imdbws.com (datasets.imdbws.com)|13.249.98.61|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 736180127 (702M) [binary/octet-stream]
Saving to: ‘title.principals.tsv.gz.4’

title.principals.ts 100%[===================>] 702.08M  42.5MB/s    in 13s     

2025-10-20 23:57:43 (54.6 MB/s) - ‘title.principals.tsv.gz.4’ saved [736180127/736180127]

--2025-10-20 23:57:43--  https://datasets.imdbws.com/title.akas.tsv.gz
Resolving datasets.imdbws.com (datasets.imdbws.com)... 13.249.98.61, 13.249.98.91, 13.249.98.73, ...
Connecting to datasets.imdbws.com (datasets.imdbws.com)|13.249.98.61|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 465498252 (444M) [binary/octet-stream]
Saving to: ‘title.akas.tsv.gz.4’

title.akas.tsv.gz.4 100%[===============

## **LOAD THE IMDB DATA**

In [ ]:
# Connect to DuckDB
con = duckdb.connect()

In [ ]:
#Title of movies
con.execute("""
CREATE TABLE title_basics AS
SELECT tconst, titleType, primaryTitle, startYear, runtimeMinutes, genres FROM read_csv_auto('title.basics.tsv.gz', delim='\t', header=true)
""")

# Title ratings
con.execute("""
CREATE TABLE title_ratings AS
SELECT * FROM read_csv_auto('title.ratings.tsv.gz', delim='\t', header=True);
""")

# Names (people)
con.execute("""
CREATE TABLE name_basics AS
SELECT nconst, primaryName, knownForTitles FROM read_csv_auto('name.basics.tsv.gz', delim='\t', header=True);
""")

#names and roles of everyone
con.execute("""
CREATE TABLE title_principals AS
SELECT tconst, nconst, category, job FROM read_csv_auto('title.principals.tsv.gz', delim='\t', header=True);
""")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

## **FEATURE ENGINEERING**

In [ ]:
#person_ratings
con.execute("""
CREATE OR REPLACE TABLE person_ratings AS
SELECT
    p.nconst,
    p.primaryName,
    ROUND(SUM(r.averageRating * r.numVotes) * 1.0 / SUM(r.numVotes), 2) AS person_rating
FROM name_basics p
JOIN title_principals tp ON p.nconst = tp.nconst
JOIN title_ratings r ON tp.tconst = r.tconst
JOIN title_basics t ON tp.tconst = t.tconst
WHERE t.titleType = 'movie'
  AND r.numVotes > 5000
  AND t.startYear != '\\N'
  AND CAST(t.startYear AS INTEGER) BETWEEN 2018 AND 2025
GROUP BY p.nconst, p.primaryName
""")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [ ]:
import duckdb
import pandas as pd

# Assuming 'con' is your DuckDB connection
df = con.execute("""
SELECT
    t.tconst,
    t.primaryTitle,
    CAST(t.startYear AS INTEGER) AS startYear,
    t.runtimeMinutes,
    t.genres,

    -- Combine actors' names and their ratings into one string
    string_agg(DISTINCT CASE WHEN tp.category = 'actor' THEN n.primaryName || ' (' || pr.person_rating || ')' END, ', ') AS actors,
    string_agg(DISTINCT CASE WHEN tp.category = 'actress' THEN n.primaryName || ' (' || pr.person_rating || ')' END, ', ') AS actresses,
    string_agg(DISTINCT CASE WHEN tp.category = 'director' THEN n.primaryName || ' (' || pr.person_rating || ')' END, ', ') AS directors,
    string_agg(DISTINCT CASE WHEN tp.category = 'writer' THEN n.primaryName || ' (' || pr.person_rating || ')' END, ', ') AS writers,
    string_agg(DISTINCT CASE WHEN tp.category = 'composer' THEN n.primaryName || ' (' || pr.person_rating || ')' END, ', ') AS composers,
    string_agg(DISTINCT CASE WHEN tp.category = 'cinematographer' THEN n.primaryName || ' (' || pr.person_rating || ')' END, ', ') AS cinematographers,
    string_agg(DISTINCT CASE WHEN tp.category = 'editor' THEN n.primaryName || ' (' || pr.person_rating || ')' END, ', ') AS editors

FROM title_basics t
JOIN title_ratings r ON t.tconst = r.tconst
JOIN title_principals tp ON t.tconst = tp.tconst
JOIN name_basics n ON tp.nconst = n.nconst
JOIN person_ratings pr ON n.nconst = pr.nconst

WHERE t.titleType = 'movie'
  AND r.numVotes > 5000
  AND t.startYear != '\\N'
  AND CAST(t.startYear AS INTEGER) BETWEEN 2018 AND 2025

GROUP BY t.tconst, t.primaryTitle, startYear, t.runtimeMinutes, t.genres

""").fetchdf()



FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [ ]:
df.to_csv('movies_ratings.csv', index=False)


In [ ]:
# Now 'final_df' is a Pandas DataFrame — view it with:
print(df.head())  # Show first 5 rows

# Or, for a nicer view in Jupyter notebooks:
df.head()

# To see the full structure:
print(df.info())

# To see summary statistics:
print(df.describe())

       tconst              primaryTitle  startYear runtimeMinutes  \
0   tt8523334               City Hunter       2018             91   
1   tt4180560                 Otherhood       2019            100   
2   tt8201852      You Should Have Left       2020             93   
3   tt1630029  Avatar: The Way of Water       2022            192   
4  tt11245972                    Scream       2022            114   

                     genres  \
0       Action,Comedy,Crime   
1                    Comedy   
2   Horror,Mystery,Thriller   
3  Action,Adventure,Fantasy   
4   Horror,Mystery,Thriller   

                                              actors  \
0  Didier Bourdon (6.5), Gérard Jugnot (6.46), Ka...   
1  Stephen Kunken (6.59), Sinqua Walls (6.32), Ja...   
2  Kevin Bacon (6.16), Colin Blumenau (5.4), Eli ...   
3  Stephen Lang (6.83), Sam Worthington (6.95), C...   
4  Dylan Minnette (5.77), David Arquette (6.24), ...   

                                           actresses  \
0  Pa

In [ ]:
df.head()

,tconst,primaryTitle,startYear,runtimeMinutes,genres,actors,actresses,directors,writers,composers,cinematographers,editors
0,tt8523334,City Hunter,2018,91,"Action,Comedy,Crime","Didier Bourdon (6.5), Gérard Jugnot (6.46), Ka...","Pamela Anderson (6.58), Élodie Fontan (6.25), ...",Philippe Lacheau (6.5),"Philippe Lacheau (6.5), Julien Arruti (6.5), T...","Michaël Tordjman (6.5), Maxime Desprez (6.5)",Vincent Richard (5.25),"Antoine Vareille (5.51), Marc David (6.5)"
1,tt4180560,Otherhood,2019,100,Comedy,"Stephen Kunken (6.59), Sinqua Walls (6.32), Ja...","Patricia Arquette (6.1), Angela Bassett (7.33)...",Cindy Chupack (6.1),"Mark Andrus (6.1), Cindy Chupack (6.1), Willia...",Marcelo Zarvos (6.8),Declan Quinn (7.8),"Sunny Hodge (6.1), Kevin Tent (7.65)"
2,tt8201852,You Should Have Left,2020,93,"Horror,Mystery,Thriller","Kevin Bacon (6.16), Colin Blumenau (5.4), Eli ...","Lowri Ann Richards (5.4), Avery Tiiu Essex (5....",David Koepp (6.23),"David Koepp (6.23), Daniel Kehlmann (5.4)",Geoff Zanelli (6.48),Angus Hudson (6.04),Derek Ambrosi (5.48)
3,tt1630029,Avatar: The Way of Water,2022,192,"Action,Adventure,Fantasy","Stephen Lang (6.83), Sam Worthington (6.95), C...","CCH Pounder (7.35), Sigourney Weaver (7.29), K...",James Cameron (7.29),"Amanda Silver (7.05), Josh Friedman (7.27), Ja...",Simon Franglen (7.38),Russell Carpenter (7.44),"David Brenner (7.68), John Refoua (7.5), Steph..."
4,tt11245972,Scream,2022,114,"Horror,Mystery,Thriller","Dylan Minnette (5.77), David Arquette (6.24), ...","Neve Campbell (6.13), Courteney Cox (6.34), Je...","Matt Bettinelli-Olpin (6.49), Tyler Gillett (6...","Kevin Williamson (6.23), James Vanderbilt (6.3...",Brian Tyler (6.39),Brett Jutkiewicz (6.63),Michel Aller (6.25)


Obtaining the all the cast average rating, director rating, writer rating, composer, editor and cinematographer ratings

In [ ]:
#person_ratings
con.execute("""
CREATE OR REPLACE TABLE person_ratings AS
SELECT
    p.nconst,
    p.primaryName,
    ROUND(SUM(r.averageRating * r.numVotes) * 1.0 / SUM(r.numVotes), 2) AS person_rating
FROM name_basics p
JOIN title_principals tp ON p.nconst = tp.nconst
JOIN title_ratings r ON tp.tconst = r.tconst
JOIN title_basics t ON tp.tconst = t.tconst
WHERE t.titleType = 'movie'
  AND r.numVotes > 5000
  AND t.startYear != '\\N'
  AND CAST(t.startYear AS INTEGER) BETWEEN 2018 AND 2025
GROUP BY p.nconst, p.primaryName

""")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [ ]:
import duckdb
import pandas as pd

# Assuming 'con' is your DuckDB connection
df = con.execute("""
SELECT
    t.tconst,
    t.primaryTitle,
    CAST(t.startYear AS INTEGER) AS startYear,
    t.runtimeMinutes,
    t.genres,
    r.averageRating AS movie_rating,

    -- Average of all actor/actress ratings
    ROUND(AVG(CASE WHEN tp.category IN ('actor', 'actress') THEN pr.person_rating END), 2) AS avg_cast_rating,

    -- Single director rating (or avg if multiple)
    ROUND(AVG(CASE WHEN tp.category = 'director' THEN pr.person_rating END), 2) AS director_rating,

    -- Same for writer and composer
    ROUND(AVG(CASE WHEN tp.category = 'writer' THEN pr.person_rating END), 2) AS writer_rating,
    ROUND(AVG(CASE WHEN tp.category = 'composer' THEN pr.person_rating END), 2) AS composer_rating,

    -- Same for cinematographer and editor
    ROUND(AVG(CASE WHEN tp.category = 'cinematographer' THEN pr.person_rating END), 2) AS cinematographer_rating,
    ROUND(AVG(CASE WHEN tp.category = 'editor' THEN pr.person_rating END), 2) AS editor_rating

FROM title_basics t
JOIN title_ratings r ON t.tconst = r.tconst
JOIN title_principals tp ON t.tconst = tp.tconst
JOIN name_basics n ON tp.nconst = n.nconst
JOIN person_ratings pr ON n.nconst = pr.nconst

WHERE t.titleType = 'movie'
  AND r.numVotes > 5000
  AND t.startYear != '\\N'
  AND CAST(t.startYear AS INTEGER) BETWEEN 2018 AND 2025

GROUP BY t.tconst, t.primaryTitle, startYear, runtimeMinutes, genres, r.averageRating


""").fetchdf()



FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [ ]:
df.to_csv('movie_rating_avg.csv', index=False)


In [ ]:
df.head()

,tconst,primaryTitle,startYear,runtimeMinutes,genres,movie_rating,avg_cast_rating,director_rating,writer_rating,composer_rating,cinematographer_rating,editor_rating
0,tt19034332,The Mystery of Marilyn Monroe: The Unheard Tapes,2022,101,"Biography,Crime,Documentary",6.2,6.20,6.20,NaN,6.54,6.20,7.33
1,tt6893836,They'll Love Me When I'm Dead,2018,98,"Biography,Documentary",7.4,NaN,7.70,NaN,5.94,7.40,7.30
2,tt4566758,Mulan,2020,115,"Action,Adventure,Drama",5.8,6.04,5.75,6.41,6.28,4.49,5.75
3,tt7131622,Once Upon a Time... in Hollywood,2019,161,"Comedy,Drama",7.6,7.35,7.60,7.60,NaN,7.13,7.39
4,tt21279806,Scoop,2024,102,"Biography,Drama",6.5,6.50,6.50,6.43,6.52,6.45,6.64
